![head.png](https://github.com/iwh-halle/FinancialDataAnalytics/blob/master/figures/head.jpg?raw=1)

# Financial Data Analytics in Python

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
MLU - Martin Luther University Halle-Wittenberg

fabian.woebbeking@iwh-halle.de

# Getting started with Alpaca paper trading

> Disclaimer: No investment advice!. I also do not endorse any software or brokerage account. Alpaca is kind enough to offer free access to their API, whether they suit your personal investment needs is for you to assess.

## Register for an Alpaca account 

1. Visit [https://alpaca.markets/](https://alpaca.markets/)
2. Creat a free account by providing your email and password 
3. After logging in, go to your "Home" dashboard 
4. On the home screen, make sure you’re in “Paper Trading” mode, where no real money is used (see screenshot below)

![Head](res/screen_home.png)

## Generate API keys 

> Do not share your API or any other credentials, hence, do not push them to GitHub! We will use our own credentials to test your code. 

1. On your home scree, scroll down until you find the section labeled "API Keys" (see screenshot below)
3. You will need to copy:
  * **API Key** 
  * **API Secret** 

![Head](res/screen_key.png)

# Test your API connection

Start by adding your API credentials into the code cell below. If you want to follow best practices in securing your credentials, you should store them in a separate .ini or .env file (which should NOT be pushed to GitHub, see .gitignore). This is, you could add a file called local.ini with the following content:

```INI
[Alpaca-Paper-Trading]
API_KEY = AddKeyHERE
API_SECRET = AddSecretHERE
```

In [1]:
import configparser
config = configparser.ConfigParser()
config.read('local.ini')
API_KEY = None
API_SECRET = None
API_KEY    = config.get('Alpaca-Paper-Trading', 'API_KEY', fallback=API_KEY)
API_SECRET = config.get('Alpaca-Paper-Trading', 'API_SECRET', fallback=API_SECRET)

In [2]:
# %pip install alpaca-py  

# Trading imports
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetOrdersRequest, GetPortfolioHistoryRequest
from alpaca.trading.enums import OrderSide, TimeInForce, QueryOrderStatus
# Data imports
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.enums import DataFeed
# Broker imports
from alpaca.broker.client   import BrokerClient

from time import sleep

In [3]:
# This is the API interface to access Alpaca's paper trading account
trading_client = TradingClient(API_KEY, API_SECRET, paper=True)

# Print account status & cash balance
account = trading_client.get_account()
print(f"Account status: {account.status}")
print(f"Cash balance:    ${account.cash}")

Account status: AccountStatus.ACTIVE
Cash balance:    $100000


# Broker basics

Before we continue, this is your primary source of information regarding Alpaca's [Alpaca-Py](https://alpaca.markets/sdks/python/) and [API](https://docs.alpaca.markets/docs/getting-started).

## Place a trade

In [9]:
# Create an order to BUY 5 shares of AAPL
order_req = MarketOrderRequest(
    symbol="AAPL",
    qty=5,
    side=OrderSide.BUY,
    time_in_force=TimeInForce.DAY
)

# Submit the order
order = trading_client.submit_order(order_data=order_req)
print("\nSubmitted order:")
print(f"  ID:     {order.id}")
print(f"  Status: {order.status}")


Submitted order:
  ID:     e436c8a2-295b-47ba-9cd6-8e17210950fb
  Status: OrderStatus.ACCEPTED


In [14]:
# Wait for the order to be processed (might need more than 5 seconds)
sleep(5)
# Pull the old order (by ID) to check its status
order = trading_client.get_order_by_id(order.id)
print("Refreshed status:", order.status)

Refreshed status: OrderStatus.ACCEPTED


## Get orders and portfolio

In [15]:
# Build orders request
get_orders_data = GetOrdersRequest(
    status=QueryOrderStatus.ALL,
    limit=500,  # Last 500 orders
    nested=True,  # include multi‐leg orders if you use options
)

# Get orders
orders = trading_client.get_orders(filter=get_orders_data)

# Print orders
print("\nOrder history:")
for order in orders:
    print(f"  ID:     {order.id}")
    print(f"  Status: {order.status}")
    print(f"  Symbol: {order.symbol}")
    print(f"  Side:   {order.side}")
    print(f"  Qty:    {order.qty}")
    print(f"  Filled: {order.filled_qty}")
    print(f"  Price:  ${order.filled_avg_price}")
    print("")


Order history:
  ID:     e436c8a2-295b-47ba-9cd6-8e17210950fb
  Status: OrderStatus.ACCEPTED
  Symbol: AAPL
  Side:   OrderSide.BUY
  Qty:    5
  Filled: 0
  Price:  $None



In [16]:
# Fetch current positions
positions = trading_client.get_all_positions()

# Print positions
print("\nPortfolio:")
if not positions:
    print("  (no open positions)")
else:
    for pos in positions:
        print(f"  {pos.symbol}: {pos.qty} shares  "
              f"(avg entry ${pos.avg_entry_price},  market ${pos.market_value})")


Portfolio:
  (no open positions)


# Obtain market data from Alpaca

In [ ]:
# This is the interface to access Alpaca's historical data
data_client = StockHistoricalDataClient(API_KEY, API_SECRET)

# Create data request, here: the last 5 one-minute bars for AAPL
bars_req = StockBarsRequest(
    symbol_or_symbols=["AAPL"],
    timeframe=TimeFrame(1, TimeFrameUnit.Minute),
    limit=5,  # Last 5 bars
    feed=DataFeed.IEX  # https://www.iex.io/
)   

# Pull the data
bars_result = data_client.get_stock_bars(bars_req)
print("\nLast 5 one-minute bars for AAPL:")
display(bars_result.df)


Last 5 one-minute bars for AAPL:


open    high     low   close  volume  \
symbol timestamp                                                           
AAPL   2025-05-06 12:56:00+00:00  197.90  197.90  197.90  197.90   200.0   
       2025-05-06 13:03:00+00:00  197.69  197.69  197.69  197.69   200.0   
       2025-05-06 13:04:00+00:00  197.63  197.64  197.63  197.64   630.0   
       2025-05-06 13:09:00+00:00  197.50  197.50  197.50  197.50   200.0   
       2025-05-06 13:14:00+00:00  197.34  197.34  197.32  197.32   600.0   

                                  trade_count        vwap  
symbol timestamp                                           
AAPL   2025-05-06 12:56:00+00:00          1.0  197.900000  
       2025-05-06 13:03:00+00:00          3.0  197.690000  
       2025-05-06 13:04:00+00:00          8.0  197.637619  
       2025-05-06 13:09:00+00:00          1.0  197.500000  
       2025-05-06 13:14:00+00:00          5.0  197.330153

In [18]:
# Reset the index so that symbol and timestamp become columns
bars = bars_result.df.reset_index()
display(bars)

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AAPL,2025-05-06 12:56:00+00:00,197.90,197.90,197.90,197.90,200.0,1.0,197.900000
1,AAPL,2025-05-06 13:03:00+00:00,197.69,197.69,197.69,197.69,200.0,3.0,197.690000
2,AAPL,2025-05-06 13:04:00+00:00,197.63,197.64,197.63,197.64,630.0,8.0,197.637619
3,AAPL,2025-05-06 13:09:00+00:00,197.50,197.50,197.50,197.50,200.0,1.0,197.500000
4,AAPL,2025-05-06 13:14:00+00:00,197.34,197.34,197.32,197.32,600.0,5.0,197.330153


In [19]:
# Pull and store historical data for MSFT
aapl_history = data_client.get_stock_bars(
    StockBarsRequest(
        symbol_or_symbols=["MSFT"],
        timeframe=TimeFrame(1, TimeFrameUnit.Day),
        start="2023-01-01",
        #end="2023-04-15",  # Optional, default is today
        feed=DataFeed.IEX,  # https://www.iex.io/
    ))
aapl_history = aapl_history.df.reset_index()
aapl_history.to_csv("data/MSFT.csv", index=False)  # Save to CSV file